In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np


In [2]:
# Depending on the machine on which the code is run, data might be stored in different directories.
# Indicate which machine is used to make sure the path to the data can be found.
# Can either be "local" or "Snellius"

MACHINE = 'local'

if MACHINE == 'Snellius':
    pred_var_path = '/gpfs/work1/0/ctdas/awoude/Ritten/predictor_vars/' # For retrieving the set of aggregated scaling vectors

    SAVE_DIR = pers_file_dir = '/gpfs/work1/0/ctdas/awoude/Ritten/trained_models/' # used for storing the trained model

    SF_DIR = '/gpfs/work1/0/ctdas/awoude/Ritten/fitted_sf/' # used for storing the scaling factor produced bij ML models

    RESULTS_DIR = '/gpfs/work1/0/ctdas/awoude/Ritten/results/' # used for storing the trained model

elif MACHINE == 'local':
    pred_var_path = './' # For retrieving the set of aggregated scaling vectors

    SAVE_DIR = pers_file_dir = './models/' # used for storing the trained model

    SF_DIR = './fitted_sf/' # used for storing the scaling factor produced bij ML models

    RESULTS_DIR = './results/'


transCom_dict = {1:'North American Boreal',
                   2:'North American Temperate',
                   7:'Eurasia Boreal',
                   8:'Eurasia Temperate',
                   11:'Europe',
                  }

In [3]:

# Loading all necessary data
with xr.open_dataset(pred_var_path + 'vars_per_eco_update.nc') as ds:
    complete_ds = ds


In [8]:
complete_ds.loc

Data variables:
    regions           (lat, lon) float32 240.0 240.0 240.0 ... 210.0 210.0 210.0
    grid_cell_weight  (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08
    grid_cell_area    (lat, lon) float32 1.079e+08 1.079e+08 ... 1.079e+08
    TER               (lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    monthly_flux      (eco_regions, time) float64 1.726e+06 ... 1.367e+05
    smoothed_flux     (eco_regions, time) float64 1.726e+06 ... 9.337e+04
    opt_flux          (eco_regions, time) float32 3.813e+05 ... 1.211e+05
    prior_flux_per_s  (eco_regions, time) float32 1.726e+06 ... 1.32e+05
    scaling_factor    (eco_regions, time) float32 146.5 231.4 ... 220.6 186.4
    sf_per_eco        (eco_regions, time) float32 0.2209 0.7194 ... 0.9176
    monthly_sf        (eco_regions, time) float64 1.0 0.2146 ... 1.036 1.035
    smoothed_sf       (eco_regions, time) float64 1.0 0.7382 ... 0.6828 0.7072
    g10m_MAX          (eco_regions, time) float64 12.99 13.69 ... 14